In [3]:
import numpy as np
import pandas as pd
import dataframe_image as dfi

# Task 1

## Raw input

In [4]:
# Using readlines()
input_steps_txt = open('/Users/michael/Documents/GitHub/AoC_2022/Day5/input_steps.txt', 'r')
Lines = input_steps_txt.readlines()
df = pd.DataFrame(columns=["Raw"])

count = 0
# Strips the newline characters
for i,line in enumerate(Lines):
    count += 1
    df.loc[i] = ([line.strip()])
    
df.head()

,Raw
0,move 3 from 4 to 6
1,move 1 from 5 to 8
2,move 3 from 7 to 3
3,move 4 from 5 to 7
4,move 1 from 7 to 8


In [5]:
# Splitting raw text into columns
df['Quantity'] = df['Raw'].str.split(pat=' ', n=-1, expand=True)[1].astype(int)
df['Target'] = df['Raw'].str.split(pat=' ', n=-1, expand=True)[3].astype(int)
df['Dest'] = df['Raw'].str.split(pat=' ', n=-1, expand=True)[5].astype(int)
df.head()

,Raw,Quantity,Target,Dest
0,move 3 from 4 to 6,3,4,6
1,move 1 from 5 to 8,1,5,8
2,move 3 from 7 to 3,3,7,3
3,move 4 from 5 to 7,4,5,7
4,move 1 from 7 to 8,1,7,8


In [6]:
df.describe()

,Quantity,Target,Dest
count,504.000000,504.000000,504.000000
mean,4.573413,5.142857,5.005952
std,4.511369,2.518999,2.528164
min,1.000000,1.000000,1.000000
25%,1.750000,3.000000,3.000000
50%,3.000000,5.000000,5.000000
75%,6.000000,7.000000,7.000000
max,33.000000,9.000000,9.000000


In [400]:
# Using readlines()
input_stacks_txt = open('/Users/michael/Documents/GitHub/AoC_2022/Day5/input_stacks_copy.txt', 'r')
Lines = input_stacks_txt.readlines()
stacks = pd.DataFrame(columns=["Raw"])

count = 0
# Strips the newline characters
for i,line in enumerate(Lines):
    count += 1
    stacks.loc[i] = ([line.strip()])
    print(line.strip())
stacks = stacks.iloc[:8]

- S C - - Z - - -
F J P - - T - N -
G H G Q - G - D -
V V D G F D - V -
R B F N N Q L S -
J M M P H V B B D
L P H D L F D J L
D T V M J N F M G


In [401]:
stacks

,Raw
0,- S C - - Z - - -
1,F J P - - T - N -
2,G H G Q - G - D -
3,V V D G F D - V -
4,R B F N N Q L S -
5,J M M P H V B B D
6,L P H D L F D J L
7,D T V M J N F M G


In [402]:
#Function for converting each row into a list
def row_list(row_num):
    row_list = stacks.iloc[row_num].str.split(' ')
    return row_list

#Collection of rows
row_index = []
for i in range(len(stacks)):
    row_index.append(row_list(i)[0])
    

In [403]:
#Making a dictionary to hold the nth value from each row to identify a stack
stack_dict = {}
for i in range(0,9):
    stack_list = []
    for x in range(0,8):
        stack_list.append(row_index[x][i])
    stack_dict[i] = stack_list

In [404]:
#Converting the dictionary to a dataframe
stack_df = pd.DataFrame(stack_dict)
stack_df

,0,1,2,3,4,5,6,7,8
0,-,S,C,-,-,Z,-,-,-
1,F,J,P,-,-,T,-,N,-
2,G,H,G,Q,-,G,-,D,-
3,V,V,D,G,F,D,-,V,-
4,R,B,F,N,N,Q,L,S,-
5,J,M,M,P,H,V,B,B,D
6,L,P,H,D,L,F,D,J,L
7,D,T,V,M,J,N,F,M,G


In [405]:
#Function to select and return a list of boxes and remove the boxes from a target column
def select_quantity(quant, target_col):
    target_stack = list(stack_df[target_col-1])
    quant_list = [i for i in range(0,quant)]
    boxes = [i for i in target_stack if i != '-']
    clean_boxes = boxes[:quant]
    for box in clean_boxes:
        box_index = target_stack.index(box)
        target_stack[box_index] = '-'
    stack_df[target_col-1] = target_stack
    return clean_boxes
            

In [406]:
#A funcion to move a list of boxes to a destination column
def move_boxes(box_list, dest_col):
    dest_col = dest_col-1
    #Length of df
    df_length = len(stack_df)
    #Lenth of dest column
    dest_col_height = len(list(stack_df[dest_col]))
    #Boxes in Dest column
    dest_col_boxes = [i for i in list(stack_df[dest_col]) if i != '-']
    
    #Scenario if destion column will exceed the length df need to increase in length
    if len(dest_col_boxes)+len(box_list)>df_length:
        for i,x in enumerate(box_list):
            stack_df.loc[-(i+1)] = ['-']*9
            stack_df.loc[-(i+1)][dest_col] = x
        stack_df.index = stack_df.index + len(box_list)
        stack_df.sort_index(inplace=True)
    
    #Scenario if the destination column if the same length as the df
    elif len(dest_col_boxes)+len(box_list)==df_length:
        box_list.reverse()
        dest_col_boxes = box_list+dest_col_boxes
        stack_df[dest_col] = dest_col_boxes
    
    #Scenario if the destination column needs empty box place holders ('-')
    else:
        empty_boxes = ['-']*(df_length-len(dest_col_boxes)-len(box_list))
        box_list.reverse()
        dest_col_boxes = empty_boxes+box_list+dest_col_boxes
        stack_df[dest_col] = dest_col_boxes
    
    


In [407]:
#Applying the functions to the stack_df to select and move boxes

for i in range(len(df)):
    quantity = df.iloc[i]['Quantity']
    target = df.iloc[i]['Target']
    dest = df.iloc[i]['Dest']
    try:
        boxes = select_quantity(quantity, target)
        move_boxes(boxes, dest)
    except:
        boxes = select_quantity(quantity, target)
        move_boxes(boxes, dest)
        
        

In [408]:
stack_df

,0,1,2,3,4,5,6,7,8
0,-,-,-,-,-,-,-,-,-
1,-,-,-,-,-,-,-,-,-
2,-,-,-,-,-,-,-,-,-
3,-,-,-,-,-,-,-,-,-
4,-,-,-,-,-,-,-,-,-
5,-,-,-,-,-,-,-,-,-
6,-,-,-,-,-,-,-,-,-
7,-,-,-,-,-,-,-,-,-
8,-,-,-,-,-,-,-,-,-
9,-,-,-,-,-,-,-,-,-


In [413]:
top_boxes = ''
for i in list(stack_df.columns):
    series = [i for i in stack_df[i] if i != '-']
    top_boxes = top_boxes+series[0]

In [414]:
top_boxes

'QMBMJDFTD'

## Task2

In [416]:
# Using readlines()
input_stacks_txt = open('/Users/michael/Documents/GitHub/AoC_2022/Day5/input_stacks_copy.txt', 'r')
Lines = input_stacks_txt.readlines()
stacks = pd.DataFrame(columns=["Raw"])

count = 0
# Strips the newline characters
for i,line in enumerate(Lines):
    count += 1
    stacks.loc[i] = ([line.strip()])
    print(line.strip())
stacks = stacks.iloc[:8]

- S C - - Z - - -
F J P - - T - N -
G H G Q - G - D -
V V D G F D - V -
R B F N N Q L S -
J M M P H V B B D
L P H D L F D J L
D T V M J N F M G


In [417]:
stacks

,Raw
0,- S C - - Z - - -
1,F J P - - T - N -
2,G H G Q - G - D -
3,V V D G F D - V -
4,R B F N N Q L S -
5,J M M P H V B B D
6,L P H D L F D J L
7,D T V M J N F M G


In [418]:
#Function for converting each row into a list
def row_list(row_num):
    row_list = stacks.iloc[row_num].str.split(' ')
    return row_list

#Collection of rows
row_index = []
for i in range(len(stacks)):
    row_index.append(row_list(i)[0])
    

In [419]:
#Making a dictionary to hold the nth value from each row to identify a stack
stack_dict = {}
for i in range(0,9):
    stack_list = []
    for x in range(0,8):
        stack_list.append(row_index[x][i])
    stack_dict[i] = stack_list

In [420]:
#Converting the dictionary to a dataframe
stack_df = pd.DataFrame(stack_dict)
stack_df

,0,1,2,3,4,5,6,7,8
0,-,S,C,-,-,Z,-,-,-
1,F,J,P,-,-,T,-,N,-
2,G,H,G,Q,-,G,-,D,-
3,V,V,D,G,F,D,-,V,-
4,R,B,F,N,N,Q,L,S,-
5,J,M,M,P,H,V,B,B,D
6,L,P,H,D,L,F,D,J,L
7,D,T,V,M,J,N,F,M,G


In [421]:
def select_quantity(quant, target_col):
    target_col = target_col-1
    target_stack = list(stack_df[target_col])
    quant_list = [i for i in range(0,quant)]
    boxes = [i for i in target_stack if i != '-']
    clean_boxes = boxes[:quant]
    for box in clean_boxes:
        box_index = target_stack.index(box)
        target_stack[box_index] = '-'
    stack_df[target_col] = target_stack
    return clean_boxes
            

In [422]:
def move_boxes(box_list, dest_col):
    dest_col = dest_col-1
    #Length of df
    df_length = len(stack_df)
    #Lenth of dest column
    dest_col_height = len(list(stack_df[dest_col]))
    #Boxes in Dest column
    dest_col_boxes = [i for i in list(stack_df[dest_col]) if i != '-']
    if len(dest_col_boxes)+len(box_list)>df_length:
        box_list.reverse()
        for i,x in enumerate(box_list):
            stack_df.loc[-(i+1)] = ['-']*9
            stack_df.loc[-(i+1)][dest_col] = x
        stack_df.index = stack_df.index + len(box_list)
        stack_df.sort_index(inplace=True)
    
    elif len(dest_col_boxes)+len(box_list)==df_length:
        box_list
        dest_col_boxes = box_list+dest_col_boxes
        stack_df[dest_col] = dest_col_boxes
    
    else:
        empty_boxes = ['-']*(df_length-len(dest_col_boxes)-len(box_list))
        box_list
        dest_col_boxes = empty_boxes+box_list+dest_col_boxes
        stack_df[dest_col] = dest_col_boxes
    
    


In [423]:
for i in range(len(df)):
    quantity = df.iloc[i]['Quantity']
    target = df.iloc[i]['Target']
    dest = df.iloc[i]['Dest']
    try:
        boxes = select_quantity(quantity, target)
        move_boxes(boxes, dest)
    except:
        print('e')
        
        

In [424]:
stack_df

,0,1,2,3,4,5,6,7,8
0,-,-,-,-,-,-,-,-,-
1,-,-,-,-,-,-,-,-,-
2,-,-,-,-,-,-,-,-,-
3,-,-,-,-,-,-,-,-,-
4,-,-,-,-,-,-,-,-,-
5,-,-,-,-,-,-,-,-,-
6,-,-,-,-,-,-,-,-,-
7,-,-,-,-,-,-,-,-,-
8,-,-,-,-,-,-,-,-,-
9,-,-,-,-,-,-,-,-,-


In [426]:
top_boxes = ''
for i in list(stack_df.columns):
    series = [i for i in stack_df[i] if i != '-']
    top_boxes = top_boxes+series[0]
top_boxes

'NBTVTJNFJ'

## Extra - saving all dfs as images to make gif

In [7]:
# Using readlines()
input_stacks_txt = open('/Users/michael/Documents/GitHub/AoC_2022/Day5/input_stacks_copy.txt', 'r')
Lines = input_stacks_txt.readlines()
stacks = pd.DataFrame(columns=["Raw"])

count = 0
# Strips the newline characters
for i,line in enumerate(Lines):
    count += 1
    stacks.loc[i] = ([line.strip()])
    print(line.strip())
stacks = stacks.iloc[:8]

- S C - - Z - - -
F J P - - T - N -
G H G Q - G - D -
V V D G F D - V -
R B F N N Q L S -
J M M P H V B B D
L P H D L F D J L
D T V M J N F M G


In [8]:
stacks

,Raw
0,- S C - - Z - - -
1,F J P - - T - N -
2,G H G Q - G - D -
3,V V D G F D - V -
4,R B F N N Q L S -
5,J M M P H V B B D
6,L P H D L F D J L
7,D T V M J N F M G


In [9]:
#Function for converting each row into a list
def row_list(row_num):
    row_list = stacks.iloc[row_num].str.split(' ')
    return row_list

#Collection of rows
row_index = []
for i in range(len(stacks)):
    row_index.append(row_list(i)[0])
    

In [10]:
#Making a dictionary to hold the nth value from each row to identify a stack
stack_dict = {}
for i in range(0,9):
    stack_list = []
    for x in range(0,8):
        stack_list.append(row_index[x][i])
    stack_dict[i] = stack_list

In [11]:
#Converting the dictionary to a dataframe
stack_df = pd.DataFrame(stack_dict)
stack_df

,0,1,2,3,4,5,6,7,8
0,-,S,C,-,-,Z,-,-,-
1,F,J,P,-,-,T,-,N,-
2,G,H,G,Q,-,G,-,D,-
3,V,V,D,G,F,D,-,V,-
4,R,B,F,N,N,Q,L,S,-
5,J,M,M,P,H,V,B,B,D
6,L,P,H,D,L,F,D,J,L
7,D,T,V,M,J,N,F,M,G


In [40]:
import os
cwd = os.getcwd()

In [12]:
def select_quantity(quant, target_col,count):
    #Exporting images
    images = cwd
    dfi.export(stack_df, images+'dataframe{}.png'.format(count))
    
    target_col = target_col-1
    target_stack = list(stack_df[target_col])
    quant_list = [i for i in range(0,quant)]
    boxes = [i for i in target_stack if i != '-']
    clean_boxes = boxes[:quant]
    for box in clean_boxes:
        box_index = target_stack.index(box)
        target_stack[box_index] = '-'
    stack_df[target_col] = target_stack
    return clean_boxes
            

In [13]:
def move_boxes(box_list, dest_col,count):
    images = cwd
    dfi.export(stack_df, images+'dataframe{}.png'.format(count))
    dest_col = dest_col-1
    #Length of df
    df_length = len(stack_df)
    #Lenth of dest column
    dest_col_height = len(list(stack_df[dest_col]))
    #Boxes in Dest column
    dest_col_boxes = [i for i in list(stack_df[dest_col]) if i != '-']
    if len(dest_col_boxes)+len(box_list)>df_length:
        box_list.reverse()
        for i,x in enumerate(box_list):
            stack_df.loc[-(i+1)] = ['-']*9
            stack_df.loc[-(i+1)][dest_col] = x
        stack_df.index = stack_df.index + len(box_list)
        stack_df.sort_index(inplace=True)
    
    elif len(dest_col_boxes)+len(box_list)==df_length:
        box_list
        dest_col_boxes = box_list+dest_col_boxes
        stack_df[dest_col] = dest_col_boxes
    
    else:
        empty_boxes = ['-']*(df_length-len(dest_col_boxes)-len(box_list))
        box_list
        dest_col_boxes = empty_boxes+box_list+dest_col_boxes
        stack_df[dest_col] = dest_col_boxes
    
    


In [ ]:
total_count=0
for i in range(len(df)):
    quantity = df.iloc[i]['Quantity']
    target = df.iloc[i]['Target']
    dest = df.iloc[i]['Dest']
    try:
        boxes = select_quantity(quantity, target,total_count)
        total_count = total_count+1
        move_boxes(boxes, dest,total_count)
        total_count = total_count+1
    except:
        print('e')
        
        